In [ ]:
# STEP 1: Install dependencies (run once)
!apt-get update -qq && apt-get install -qq imagemagick
!sed -i 's/<policy domain="coder" rights="none" pattern="TXT" \/>/<policy domain="coder" rights="read|write" pattern="TXT" \/>/' /etc/ImageMagick-6/policy.xml
!pip install -q google-generativeai gTTS moviepy requests Pillow python-dotenv

# STEP 2: Imports
import os
import requests
import google.generativeai as genai
from gtts import gTTS
from moviepy.editor import *
from PIL import Image, ImageDraw, ImageFont
import numpy as np
from dotenv import load_dotenv

# STEP 3: Configure Gemini API Key from .env
load_dotenv()  # Loads variables from .env
GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")
if not GEMINI_API_KEY:
    raise ValueError("Gemini API Key not found! Please set GEMINI_API_KEY in your .env file.")
genai.configure(api_key=GEMINI_API_KEY)

# STEP 4: Generate AI Script from Topic
def generate_script(topic):
    model = genai.GenerativeModel('models/gemini-1.5-flash')
    prompt = f"Write an engaging 6-8 line YouTube Shorts script on the topic: '{topic}'. Keep it short, clear, and catchy."
    response = model.generate_content(prompt)
    return response.text

# STEP 5: Download images function (with fallback)
def download_images(keywords):
    os.makedirs("images", exist_ok=True)
    headers = {"User-Agent": "Mozilla/5.0"}
    for i, kw in enumerate(keywords):
        try:
            url = f"https://source.unsplash.com/720x480/?{kw.replace(' ', '%20')}"
            r = requests.get(url, headers=headers, timeout=10)
            with open(f"images/img{i+1}.jpg", "wb") as f:
                f.write(r.content)
        except:
            img = Image.new('RGB', (720, 480), color='gray')
            img.save(f"images/img{i+1}.jpg")

# STEP 6: Create TTS audio clip per line
def create_tts_audio(line, idx):
    filename = f"voice_line_{idx}.mp3"
    tts = gTTS(text=line, lang='en')
    tts.save(filename)
    return AudioFileClip(filename)

# STEP 7: Create video clip with image + text overlay
def create_clip(line, img_path, duration):
    try:
        bg = Image.open(img_path).resize((720, 480)).convert('RGB')
    except:
        bg = Image.new('RGB', (720, 480), color='gray')
    draw = ImageDraw.Draw(bg)
    try:
        font = ImageFont.truetype("DejaVuSans-Bold.ttf", 36)
    except:
        font = ImageFont.load_default()
    bbox = draw.textbbox((0, 0), line, font=font)
    w = bbox[2] - bbox[0]
    h = bbox[3] - bbox[1]
    position = ((720 - w)//2, 480 - h - 40)
    draw.text(position, line, font=font, fill='white', stroke_width=2, stroke_fill='black')

    frame = np.array(bg).astype(np.uint8)
    clip = ImageClip(frame).set_duration(duration)
    return clip

# STEP 8: Optional background music loader
def load_background_music(path, duration):
    if not os.path.exists(path):
        print(f"Background music file '{path}' not found, skipping bgm.")
        return None
    bgm = AudioFileClip(path).volumex(0.15)
    if bgm.duration < duration:
        bgm = afx.audio_loop(bgm, duration=duration)
    else:
        bgm = bgm.subclip(0, duration)
    return bgm

# ===== MAIN SCRIPT =====

topic = "The Rise of Electric Vehicles"  # Change your topic here

print("Generating script...")
script_text = generate_script(topic)
print("Generated script:\n", script_text)

lines = [line.strip() for line in script_text.split('\n') if line.strip()]

print("Downloading images...")
download_images(lines)

print("Generating audio clips...")
audio_clips = []
for i, line in enumerate(lines):
    audio = create_tts_audio(line, i)
    audio_clips.append(audio)

print("Creating video clips...")
video_clips = []
for i, (line, audio) in enumerate(zip(lines, audio_clips)):
    clip = create_clip(line, f"images/img{i+1}.jpg", duration=audio.duration)
    clip = clip.set_audio(audio)
    video_clips.append(clip)

print("Concatenating clips...")
final_video = concatenate_videoclips(video_clips, method="compose")

# Uncomment below to add background music
# !wget -q -O bgm.mp3 https://files.freemusicarchive.org/storage-freemusicarchive-org/music/no_curator/Komiku/Its_time_for_adventure/Komiku_-_12_-_The_time_is_now.mp3
# bgm = load_background_music("bgm.mp3", final_video.duration)
# if bgm:
#     final_audio = CompositeAudioClip([final_video.audio, bgm])
#     final_video = final_video.set_audio(final_audio)

output_filename = "final_ev_video.mp4"
print(f"Exporting final video to {output_filename} ...")
final_video.write_videofile(output_filename, fps=24, codec="libx264", audio_codec="aac")

# Cleanup
for i in range(len(lines)):
    fname = f"voice_line_{i}.mp3"
    if os.path.exists(fname):
        os.remove(fname)

print("Done! Your video is ready.")
from IPython.display import Video
Video(output_filename, embed=True)
